In [31]:
# cd.. In separate cell if used

from lib.network_mapper import network_mapper_function
from lib.NR3 import NR3_function
#from lib.computeFTreal import computeFTrealfunction
#from lib.computeJTreal import computeJTrealfunction
#from lib import *
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
#matplotlib inline
fig_size = np.array([18, 6])
plt.rcParams["figure.figsize"] = fig_size

# z = exp(s*T)

# zoh
# z ~ 1 + s*T
# s ~ (z - 1)/T

# bilinear transform
# z ~ (2 + sT)/(2 - sT)
# s ~ (2/T)(z - 1)/(z + 1)

In [32]:
fp = "C:\\Users\\kathl\\Desktop\\LinDist3Flow\\20180601\\NETWORKS\\"

fn = '05node_singlephase_radial.txt';
fn = '05node_singlephase_radial.txt'
fn = "03node_singlephase_radial_example.txt"

#fn = "03node_fullphase_radial_example.txt"
#fn = '03node_fullphase_mesh_example.txt' #works
#fn = 'ieee_13node_mesh_open.txt' #works
#fn = '05node_fullphase_radial.txt'; #works

#fn = 'ieee_13node_balance.txt' #works except sNr and iNR are off; check again
fn = 'ieee_13node_mesh_open.txt'; #works
fn = 'ieee_34node.txt' #iNR and sNR are off

network1 = network_mapper_function(fp, fn)

nnode1 = network1.nodes.nnode
nline1 = network1.lines.nline

# Network paramaters

nnode = network1.nodes.nnode
nline = network1.lines.nline

# Load parameters

network1.loads.aPQ = 1.00*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aI = 0.00*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aZ = 0.00*np.ones((3,nnode1))*(network1.loads.spu != 0)

network1.loads.aPQ = .750*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aI = 0.10*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aZ = 0.15*np.ones((3,nnode1))*(network1.loads.spu != 0)


network1.loads.spu = 1.0*network1.loads.spu
# Capacitor parameters

#network1.caps.cappu = 1*network1.caps.cappu

# Controller parameters

#network1.cons.wmaxpu = 0.5*network1.cons.wmaxpu


BASE
Vbase: 24900.0
Sbase: 2500000.0
Ibase: 100.40160642570281
Zbase: 248.00400000000002

NODES
nnode: 35
nodelist: ['SUB', '800', '802', '806', '808', '810', '812', '814', '850', '816', '818', '820', '822', '824', '826', '828', '830', '854', '856', '852', '832', '888', '890', '858', '864', '834', '842', '844', '846', '848', '860', '836', '840', '862', '838']
phases: ['abc', 'abc', 'abc', 'abc', 'abc', 'b', 'abc', 'abc', 'abc', 'abc', 'a', 'a', 'a', 'abc', 'b', 'abc', 'abc', 'abc', 'b', 'abc', 'abc', 'abc', 'abc', 'abc', 'a', 'abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'c']
PH:
 [[1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0]
 [1 1 1 1 1 0 1 1 1 1 0 0 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1]]

LINES
nline: 34
TXnode: ['SUB', '800', '802', '806', '808', '808', '812', '814', '850', '816', '818', '820', '816', '824', '824', '828', '830', '854', '854', '852', '832', '8

In [33]:
#Count the number of phases that exist
phase_count = 0;
a = 0;
b = 0;
c = 0;
for phaseCount in range(len(network1.nodes.phases)):
    curr = network1.nodes.phases[phaseCount];
    if 'a' in curr and a == 0:
        phase_count = phase_count + 1
        a = 1
    if 'b' in curr and b == 0:
        b = 1
        phase_count = phase_count + 1
    if 'c' in curr and c == 0:
        phase_count = phase_count+1
        c = 1

        
csv_file_count = 1;
all_demand = np.zeros((3, nnode, 1441))
#Assign each node and phase a loading profile via all_demand
for ph in range(3):
    for kn in range(nnode):
        if (a == 1 and ph == 0) or (b == 1 and ph == 1) or (c == 1 and ph == 2):
            if csv_file_count < 38:
                csvfile = pd.read_csv(r'C:\Users\kathl\Desktop\LinDist3Flow\20180601\testpvnum0\node_'+str(csv_file_count)+'_pv_0_minute_normalized.csv')
            elif csv_file_count < 75 and csv_file_count > 37:
                
                csvfile = pd.read_csv(r'C:\Users\kathl\Desktop\LinDist3Flow\20180601\testpvnum1\node_'+str((csv_file_count % 38) + 1)+'_pv_1_minute_normalized.csv')
            elif csv_file_count > 76 and csv_file_count < 105:
              
                csvfile = pd.read_csv(r'C:\Users\kathl\Desktop\LinDist3Flow\20180601\testpvnum2\node_'+str((csv_file_count % 37) + 1)+'_pv_2_minute_normalized.csv')
            else:
                csvfile = pd.read_csv(r'C:\Users\kathl\Desktop\LinDist3Flow\20180601\testpvnum3\node_'+str((csv_file_count % 37) + 1)+'_pv_3_minute_normalized.csv')
            
            demand = csvfile.iloc[:, 5]
            vals = demand.values
            all_demand[ph,kn] = vals
            csv_file_count = csv_file_count + 1
            

In [34]:
phstr = ['a','b','c']

# Recompute spu nominal based on the node and phase's loading profile
for kt in range(50): #Run for 50 time steps
    for ph in range(3): #3 phases
        for kn in range(nnode): #nnodes
            if (a == 1 and ph == 0) or (b == 1 and ph == 1) or (c == 1 and ph == 2):
                network1.loads.spu_nominal[ph,kn] = network1.loads.spu[ph,kn]*all_demand[ph,kn,kt]
              

    network1.cons.wpu = np.zeros((3,nnode))
    network1.vvc.vvcpu = np.zeros((3,nnode))

    slackidx = 0
    Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])
    
    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_function(network1,slackidx,Vslack,None,None,None,None)

    fid = open('NR3-python.txt','a')
 
    fid.write('VNR\n\n')
    for ph in range(0,3):
        wstr = phstr[ph] + '\n'
        fid.write(wstr)
        for k1 in range(0,nnode):
            wstr = '{0:.6f}'.format(VNR[ph,k1].real) + ' ' + '{0:.6f}'.format(VNR[ph,k1].imag) + '\n'
            fid.write(wstr)

    fid.write('\n\nINR\n\n')
    for ph in range(0,3):
        wstr = phstr[ph] + '\n'
        fid.write(wstr)
        for k1 in range(0,nline):
            wstr = '{0:.6f}'.format(INR[ph,k1].real) + ' ' + '{0:.6f}'.format(INR[ph,k1].imag) + '\n'
            fid.write(wstr)

    fid.write('\n\nSTXNR\n\n')
    for ph in range(0,3):
        wstr = phstr[ph] + '\n'
        fid.write(wstr)
        for k1 in range(0,nline):
            wstr = '{0:.6f}'.format(STXNR[ph,k1].real) + ' ' + '{0:.6f}'.format(STXNR[ph,k1].imag) + '\n'
            fid.write(wstr)

    fid.write('\n\nSRXNR\n\n')
    for ph in range(0,3):
        wstr = phstr[ph] + '\n'
        fid.write(wstr)
        for k1 in range(0,nline):
            wstr = '{0:.6f}'.format(SRXNR[ph,k1].real) + ' ' + '{0:.6f}'.format(SRXNR[ph,k1].imag) + '\n'
            fid.write(wstr)

    fid.write('\n\niNR\n\n')
    for ph in range(0,3):
        wstr = phstr[ph] + '\n'
        fid.write(wstr)
        for k1 in range(0,nnode):
            wstr = '{0:.6f}'.format(iNR[ph,k1].real) + ' ' + '{0:.6f}'.format(iNR[ph,k1].imag) + '\n'
            fid.write(wstr)

    fid.write('\n\nsNR\n\n')
    for ph in range(0,3):
        wstr = phstr[ph] + '\n'
        fid.write(wstr)
        for k1 in range(0,nnode):
            wstr = '{0:.6f}'.format(sNR[ph,k1].real) + ' ' + '{0:.6f}'.format(sNR[ph,k1].imag) + '\n'
            fid.write(wstr)

    fid.close()

In [ ]:
network1.cons.wpu = np.zeros((3,nnode))
network1.vvc.vvcpu = np.zeros((3,nnode))

slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])


VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
    NR3_function(network1,slackidx,Vslack,None,None,None,None)

'''
VNR = np.round(VNR,decimals=6)
INR = np.round(INR,decimals=6)
STXNR = np.round(STXNR,decimals=6)
SRXNR = np.round(SRXNR,decimals=6)
iNR = np.round(iNR,decimals=6)
sNR = np.round(sNR,decimals=6)
'''

print('VNR\n', np.round(VNR,decimals=6))
print('INR\n', np.round(INR,decimals=6))
print('STXNR\n', np.round(STXNR,decimals=6))
print('SRXNR\n', np.round(SRXNR,decimals=6))
print('iNR\n', np.round(iNR,decimals=6))
print('sNR\n', np.round(sNR,decimals=6))
print('itercount:', itercount)

print('|VNR|\n', np.round(np.abs(VNR),decimals=6))
print('<VNR\n', np.round(180/np.pi*np.angle(VNR),decimals=6))
print('D<VNR\n', 180/np.pi*np.angle(VNR) - 180/np.pi*np.angle(VNR[:,[0]]))

In [ ]:
fid = open('NR3-' + fn[:-4] + '-python.txt','w')

phstr = ['a','b','c']

fid.write('VNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nnode):
        wstr = '{0:.6f}'.format(VNR[ph,k1].real) + ' ' + '{0:.6f}'.format(VNR[ph,k1].imag) + '\n'
        fid.write(wstr)

fid.write('\n\nINR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nline):
        wstr = '{0:.6f}'.format(INR[ph,k1].real) + ' ' + '{0:.6f}'.format(INR[ph,k1].imag) + '\n'
        fid.write(wstr)
        
fid.write('\n\nSTXNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nline):
        wstr = '{0:.6f}'.format(STXNR[ph,k1].real) + ' ' + '{0:.6f}'.format(STXNR[ph,k1].imag) + '\n'
        fid.write(wstr)

fid.write('\n\nSRXNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nline):
        wstr = '{0:.6f}'.format(SRXNR[ph,k1].real) + ' ' + '{0:.6f}'.format(SRXNR[ph,k1].imag) + '\n'
        fid.write(wstr)
        
fid.write('\n\niNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nnode):
        wstr = '{0:.6f}'.format(iNR[ph,k1].real) + ' ' + '{0:.6f}'.format(iNR[ph,k1].imag) + '\n'
        fid.write(wstr)
        
fid.write('\n\nsNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nnode):
        wstr = '{0:.6f}'.format(sNR[ph,k1].real) + ' ' + '{0:.6f}'.format(sNR[ph,k1].imag) + '\n'
        fid.write(wstr)

fid.close()